In [1]:
from pathlib import Path
import pandas as pd

In [2]:
run = Path('/mnt/nfs/home/Florian.Borchert/runs/xmen/symptemist_benchmark/outputs/2024-02-07_16-15-59')

In [3]:
base_path = Path.home() / '.cache' / 'xmen' / 'symptemist'

In [4]:
from xmen import load_kb
kb = load_kb(base_path / 'symptemist.jsonl')

In [5]:
len(kb.alias_to_cuis)

992924

In [ ]:
sum([len(v) for a, v in kb.alias_to_cuis.items()])

In [ ]:
list(run.glob('*'))

In [ ]:
import datasets

In [ ]:
dataset = datasets.load_from_disk(run / '0-symptemist_benchmark_dataset')
candidates = datasets.load_from_disk(run / '0-symptemist_benchmark_candidates')
pred_val =  datasets.load_from_disk(run / '0-symptemist_benchmark_cross_enc_pred_val')
pred_val_no_nil =  datasets.load_from_disk(run / '0-symptemist_benchmark_cross_enc_pred_val_no_nil')
pred_test = datasets.load_from_disk(run / '0-symptemist_benchmark_cross_enc_pred_test')
pred_test_no_nil = datasets.load_from_disk(run / '0-symptemist_benchmark_cross_enc_pred_test_no_nil')

In [ ]:
from xmen.evaluation import evaluate, error_analysis, evaluate_at_k

In [ ]:
_ = evaluate_at_k(dataset['validation'], candidates['validation'])

In [ ]:
evaluate(dataset['test'], pred_test_no_nil)

In [ ]:
ea = error_analysis(dataset['validation'], candidates['validation'])

In [ ]:
errors = ea[ea.pred_index == -1]

In [ ]:
ea1 = ea[ea.pred_index == 0]#._word_len.value_counts().sort_index()
ea2 = ea[ea.pred_index == -1]#._word_len.value_counts().sort_index()

In [ ]:
ea[ea.pred_index == -1]

In [ ]:
(ea1._word_len.value_counts() / len(ea1)).sort_index().plot()
(ea2._word_len.value_counts() / len(ea2)).sort_index().plot()

# Lookup

In [ ]:
str2cui = []
for d in dataset['train']['entities']:
    for e in d:
        str2cui.append({'text' : ' '.join(e['text']), 'norm' : e['normalized'][0]['db_id']})
str2cui = pd.DataFrame(str2cui)
str2norm = str2cui.groupby('text').agg(set).norm

str2cui_list = str2cui.groupby('text').agg(list).norm

lookup = str2cui_list[str2cui_list.map(len) > 1].map(lambda l: l[0])
lookup = str2norm.map(lambda s: list(s)[0])

# Mapping is unique
str2norm[str2norm.map(len) != 1]

def transform_lookup(sample):
    entities = sample['entities'].copy()
    for e in entities:
        t = ' '.join(e['text'])
        if t in lookup.index:
            train_cui = lookup.loc[t]
            norm = e['normalized']
            entry = {'db_name': 'SNOMED_CT', 'db_id': train_cui, 'score' : 1.0, 'predicted_by' : ['lookup']}
            norm.insert(0, entry)
    return { 'entities' : entities }

In [ ]:
evaluate(dataset['validation'], pred_val)

In [ ]:
evaluate(dataset['validation'], pred_val.map(transform_lookup))